In [4]:
import kivy
from kivy.app import App 
from kivy.uix.floatlayout import FloatLayout 
from kivy.uix.boxlayout import BoxLayout 
from kivy.uix.label import Label 
from kivy.uix.button import Button 
from kivy.uix.filechooser import FileChooserIconView
from kivy.uix.filechooser import FileChooserListView
from kivy.uix.filechooser import FileChooserLayout

from kivy.properties import ObjectProperty 
from kivy.uix.popup import Popup 
from kivy.factory import Factory 
from kivy.uix.screenmanager import Screen,ScreenManager # from kivy.clock import Clock 
from kivy.graphics.texture import Texture 
from kivy.uix.camera import Camera 
from kivy.properties import BooleanProperty, NumericProperty 
import os 
import cv2 
import numpy as np 
import logging 
from kivy.core.window import Window
#from sample_lane_detection import *

kivy.require('2.0.0')

#indexPage
class IndexPage(FloatLayout):

    def __init__(self,**kwargs):
        super().__init__(**kwargs)
    def page_go(*args):
        App.get_running_app().screen_manager.current="Image_page"
        App.get_running_app().screen_manager.transition.direction = 'left'#这个是过长动画的方向


def detect_img(img):
    #print(img)
    return img
def cv_imread(filePath):
    with open(filePath,mode="rb") as f:
        img_buff = np.frombuffer(f.read(),dtype=np.uint8)
    cv_img = cv2.imdecode(img_buff,-1)
    return cv_img
def kivy_showcvimg(img,win):
    img_buff = img.tostring()
    img_texture = Texture.create(size=(img.shape[1], img.shape[0]), colorfmt='rgba')
    img_texture.blit_buffer(img_buff, colorfmt='rgba', bufferfmt='ubyte')
    win.texture = img_texture
#Image page
class LoadDialog(FloatLayout):
    load = ObjectProperty(None)
    cancel = ObjectProperty(None)
    cwdir = ObjectProperty(None)
class LoadFolder(FloatLayout):
    load = ObjectProperty(None)
    cancel = ObjectProperty(None)
    cwdir = ObjectProperty(None)
Factory.register("LoadDialog",cls=LoadDialog)#注册一个类？

class ImagePage(BoxLayout):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
    def selectfolder(self):
        print("selectfolder")
        content = LoadFolder(load=self.openfile,cancel=self.dismiss_popup,cwdir=os.getcwd())
        self._popup = Popup(title="Load Folder",content=content,size_hint=(.9,.9))
        self._popup.open()
    def openfile(self,path,filename):
        print(path,filename)
    def back_index(*args):
        App.get_running_app().screen_manager.current="Index_page"#这个页面返回是Index_page
        App.get_running_app().screen_manager.transition.direction = 'right'
    def back_video(self,*args):
        App.get_running_app().screen_manager.current="Video_page"
        App.get_running_app().screen_manager.transition.direction = 'left'
    def dismiss_popup(self):
        self._popup.dismiss()
    def show_load(self):
        content = LoadDialog(load=self._load,cancel=self.dismiss_popup,cwdir=os.getcwd())
        self._popup = Popup(title="Load Image",content=content,size_hint=(.9,.9))
        self._popup.open()
    def _load(self,path,filename):
        print(path,filename)
        self.dismiss_popup()
        logging.info("path:{},filename:{}".format(path,filename))
        print("filename")
        img = cv_imread(filename)
        img = detect_img(img)
        print(img.shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
        #cv2.imshow(img)
        img = cv2.flip(img, 0) # 旋转
        #cv2.imshow(img)
        print("after flip",img.shape)
        kivy_showcvimg(img,self.ids.img_det)
        kivy_showcvimg(img,self.ids.img_det1)
        #img_buff = img.tostring()
        #img_texture = Texture.create(size=(img.shape[1], img.shape[0]), colorfmt='rgba')
        #img_texture.blit_buffer(img_buff, colorfmt='rgba', bufferfmt='ubyte')
        #self.ids.img_det.texture = img_texture
        

class LaneDetectApp(App):
    def build(self):
        Window.fullscreen=0
        Window.size=(1000,400)
        self.icon = "./static/icon.ico" 
        self.title = "智能车道线检测App"
        self.load_kv("./index.kv") # 需要创建一个index.kv 
        self.load_kv("./image.kv")
        self.screen_manager = ScreenManager()
        pages = {"Index_page":IndexPage(),"Image_page":ImagePage(),}

        for item,page in pages.items(): 
            print(item)
            self.default_page = page
             # 添加页面
            screen = Screen(name=item)
            screen.add_widget(self.default_page) # 向屏幕管理器添加页面
            self.screen_manager.add_widget(screen)
        return self.screen_manager


LaneDetectApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Administrator\.kivy\logs\kivy_22-05-24_57.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.gstreamer" 0.3.2
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.3.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.3.1
[INFO   ] [Kivy        ] v2.0.0
[INFO   ] [Kivy        ] Installed at "D:\anaconda\envs\OpenPose\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "D:\anaconda\envs\OpenPose\python.exe"
[INFO   ] [Factory     ] 186 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Camera      ] Provider: opencv(['camera_picamera', 'camera_gi'] ignored)
[INFO   ] [Window      ] Provide

NameError: name 'kivy' is not defined

In [11]:
def GetFoderImagePaths(path):
    import os
    L=[]
    N=[]
    for root,dirs,files in os.walk(path):
        for file in files:
            if os.path.splitext(file)[1] == '.jpg':
                L.append(os.path.join(root,file))
                N.append(file)
    return L,N


In [12]:
paths,fnames = GetFoderImagePaths("C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\")

In [13]:
print(paths,fnames)

['C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (1).jpg', 'C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (2).jpg', 'C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (3).jpg', 'C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (4).jpg', 'C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (5).jpg', 'C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (6).jpg', 'C:\\Users\\Administrator\\Documents\\mydemos\\PyGUI\\TestImages\\DayTable (7).jpg'] ['DayTable (1).jpg', 'DayTable (2).jpg', 'DayTable (3).jpg', 'DayTable (4).jpg', 'DayTable (5).jpg', 'DayTable (6).jpg', 'DayTable (7).jpg']


In [14]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [15]:
#import algo as al
import pandas as pd
def ReadAndReFormCSV(path):
    df=pd.read_csv(path)
    cname=df.columns[0]
    df.index=df[cname]
    df=df.drop(cname,axis=1)
    return df

In [21]:
#判断observer.csv是否存在
import os
if(os.path.exists("observer.csv")==False):
    df = pd.DataFrame([["nobody",0]],columns=["observer","observerid"])
    df.to_csv("observer.csv")

In [ ]:
#想observer添加一个observer
def AddOneObserver(newName):
    if(os.path.exists("observer.csv")==False):
        df = pd.DataFrame([["nobody",0]],columns=["observer","observerid"])
        df.to_csv("observer.csv")
    else:
        df = pd.read_csv("observer.csv")
        df = pd.DataFrame([["nobody",0]],columns=["observer","observerid"])
        